In [1]:
import pandas as pd
import numpy as np
from dataset_functions import *
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
df,target = get_dataset()
df = teamstats_teamheros_transform(df)
df = team_weighted_mean_position_transform(df)
df = feature_selection_transform(df, target['radiant_win'], 0.01)


df_test = get_test_dataset()
df_test = teamstats_teamheros_transform(df_test)
df_test = team_weighted_mean_position_transform(df_test)
df_test = df_test[df.columns] 

Filtering Df:  (game_mode == 2 or game_mode == 22) and game_time > 0 

Dropped:  ['lobby_type', 'chat_len', 'game_mode', 'match_id_hash'] 

Dataframe Shape:  (32153, 242) 

Target shape: (32153, 6)
Hero Id Labels: ['r1_hero_id', 'r2_hero_id', 'r3_hero_id', 'r4_hero_id', 'r5_hero_id', 'd1_hero_id', 'd2_hero_id', 'd3_hero_id', 'd4_hero_id', 'd5_hero_id'] 

Numbers of Heros:  115 

NaN Count:  0 

Single Player Labels: ['r1_kills', 'r1_deaths', 'r1_assists', 'r1_denies', 'r1_gold', 'r1_lh', 'r1_xp', 'r1_health', 'r1_max_health', 'r1_max_mana', 'r1_level', 'r1_x', 'r1_y', 'r1_stuns', 'r1_creeps_stacked', 'r1_camps_stacked', 'r1_rune_pickups', 'r1_firstblood_claimed', 'r1_teamfight_participation', 'r1_towers_killed', 'r1_roshans_killed', 'r1_obs_placed', 'r1_sen_placed', 'r2_kills', 'r2_deaths', 'r2_assists', 'r2_denies', 'r2_gold', 'r2_lh', 'r2_xp', 'r2_health', 'r2_max_health', 'r2_max_mana', 'r2_level', 'r2_x', 'r2_y', 'r2_stuns', 'r2_creeps_stacked', 'r2_camps_stacked', 'r2_rune_pickups

In [3]:
model = HistGradientBoostingClassifier(l2_regularization=0.1, max_features=0.5, min_samples_leaf=50, max_iter=200,validation_fraction=0.05,scoring="roc_auc")
model.fit(df,target["radiant_win"])

HistGradientBoostingClassifier(l2_regularization=0.1, max_features=0.5,
                               max_iter=200, min_samples_leaf=50,
                               scoring='roc_auc', validation_fraction=0.05)

In [8]:
predictions = model.predict_proba(df_test)[:,1]
predictions

array([0.54718744, 0.98487916, 0.94613742, ..., 0.29578198, 0.73750446,
       0.06454642])

In [12]:
featureset_path = "../dataset/mlcourse-dota2-win-prediction/test_features.csv"
df_test = pd.read_csv(featureset_path)

predictions_df = df_test[["match_id_hash"]].copy()
predictions_df["radiant_win_prob"] = predictions
predictions_df.to_csv("../submission.csv", index=False)